In [14]:
import random
import pandas as pd
import gym
import stable_baselines3
from gym import spaces
import numpy as np
from stable_baselines3 import PPO,DQN,A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import CheckpointCallback


In [3]:
df = pd.read_csv(r'C:\Users\user\Desktop\123244\binance_data.csv')

In [ ]:
df

In [69]:
class stablebaselineEnv(gym.Env):
    def __init__(self, df, full_window_size, test_window_size, usdt_balance, btc_size=0, leverage=1): 
        super(stablebaselineEnv, self).__init__()
        self.slice_df, self.obs_df, self.train_df = stablebaselineEnv.generate_random_data_slice(df, full_window_size, test_window_size) # 랜덤 위치로 slice된 차트 데이터 초기화
        self.action_space = spaces.Discrete(4)  # 0: Long, 1: Short, 2: Close, 3: Hold
        self.current_step = self.slice_df.tail(1)
        self.observation_space = spaces.Dict({
            "chart_data": spaces.Box(low=0, high=np.inf, shape=(len(self.current_step.columns),), dtype=np.float32), # 차트 데이터
            "position": spaces.Discrete(3),  # 포지션 {0:Long, 1:Short, 2:None}
            "action": spaces.Discrete(4),  # 액션 {0:Long, 1:Short, 2:Close, 3:Hold}
            "current_price": spaces.Box(low=0, high=np.inf, shape=(), dtype=np.float32),  # 현재 가격
            "avg_price": spaces.Box(low=0, high=np.inf, shape=(), dtype=np.float32),  # 평균 진입 가격
            "pnl": spaces.Box(low=-np.inf, high=np.inf, shape=(), dtype=np.float32),  # 미실현 손익
            "total_pnl": spaces.Box(low=-np.inf, high=np.inf, shape=(), dtype=np.float32),  # 누적 손익
            "usdt_balance": spaces.Box(low=0, high=np.inf, shape=(), dtype=np.float32),  # USDT 잔고
            "size": spaces.Box(low=0, high=np.inf, shape=(), dtype=np.float32),  # 포지션 수량
            "margin": spaces.Box(low=0, high=np.inf, shape=(), dtype=np.float32),  # 사용 중인 마진
            "total_balance": spaces.Box(low=0, high=np.inf, shape=(), dtype=np.float32)  # 총 자산
        })
        self.full_window_size = full_window_size
        self.test_window_size = test_window_size
        self.start_step = self.full_window_size
        self.window_size = self.full_window_size
        self.current_price = 0


        # reset 미포함
        self.initial_usdt_balance = usdt_balance # 초기 usdt 잔고
        self.min_order = 0.002 # 최소 주문 수량
        self.fee = 0.0005 # 거래 수수료
        self.leverage = leverage # 레버리지
        
        # reset 포함
        self.usdt_balance = usdt_balance # 초기 usdt 잔고
        self.btc_size = btc_size # 포지션 수량
        self.margin = 0 # 포지션 증거금
        self.position = 2 # 포지션 {0:Long, 1:Short, 2:None}
        self.order_price = 0 # 주문 금액
        self.last_size_value = 0 # (평단가 계산에 필요)
        self.current_avg_price = 0 # 현재 평단가
        self.pnl = 0 # 미실현 손익
        self.closing_pnl = 0 # 실현 손익
        self.total_pnl = 0 # 누적 손익
        self.total_fee = 0 # 누적 수수료
        self.total_balance = 0 # 총 자산
        self.action_history = pd.DataFrame(columns=['action'])
        self.current_index = 0
        pass

    def reset(self): # 리셋 함수 
        self.slice_df, self.obs_df, self.train_df = stablebaselineEnv.generate_random_data_slice(df, self.full_window_size, self.test_window_size) # 랜덤 위치로 slice된 차트 데이터 초기화
        self.action_space = spaces.Discrete(4)  # 0: Long, 1: Short, 2: Close, 3: Hold
        self.current_step = self.slice_df.tail(1)
        self.observation_space = spaces.Dict({
            "chart_data": spaces.Box(low=0, high=np.inf, shape=(len(self.current_step.columns),), dtype=np.float32), # 차트 데이터
            "position": spaces.Discrete(3),  # 포지션 {0:Long, 1:Short, 2:None}
            "action": spaces.Discrete(4),  # 액션 {0:Long, 1:Short, 2:Close, 3:Hold}
            "current_price": spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),  # 현재 가격
            "avg_price": spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),  # 평균 진입 가격
            "pnl": spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32),  # 미실현 손익
            "total_pnl": spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32),  # 누적 손익
            "usdt_balance": spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),  # USDT 잔고
            "size": spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),  # 포지션 수량
            "margin": spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),  # 사용 중인 마진
            "total_balance": spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32)  # 총 자산
        })

        self.start_step = self.full_window_size
        self.window_size = self.full_window_size
        self.current_price = 0
        self.usdt_balance = self.initial_usdt_balance # 초기 usdt 잔고
        self.btc_size = 0 # 포지션 수량
        self.margin = 0 # 포지션 증거금
        self.position = 2 # 포지션 {0:Long, 1:Short, 2:None}
        self.order_price = 0 # 주문 금액
        self.last_size_value = 0 # (평단가 계산에 필요)
        self.current_avg_price = 0 # 현재 평단가
        self.pnl = 0 # 미실현 손익
        self.closing_pnl = 0 # 실현 손익
        self.total_pnl = 0 # 누적 손익
        self.total_fee = 0 # 누적 수수료
        self.total_balance = 0 # 총 자산
        self.action_history = pd.DataFrame(columns=['action'])
        self.current_index = 0
        
        # obs = {
        #     "chart_data": self.current_step.values.flatten(),
        #     "position": self.position,
        #     "action": 3,  # Hold 액션으로 초기화
        #     "current_price": self.current_price,
        #     "avg_price": self.current_avg_price,
        #     "pnl": self.pnl,
        #     "total_pnl": self.total_pnl,
        #     "usdt_balance": round(self.usdt_balance, 2),
        #     "size": self.btc_size,
        #     "margin": self.margin,
        #     "total_balance": self.total_balance
        # }
        obs = {
            "chart_data": self.current_step.values,  # 차원 조정: 2차원 배열로 변환
            "position": np.array([self.position]),  # 차원 조정: 1차원 배열로 변환
            "action": np.array([3]),  # Hold 액션으로 초기화, 차원 조정: 1차원 배열로 변환
            "current_price": np.array([self.current_price]),  # 차원 조정: 1차원 배열로 변환
            "avg_price": np.array([self.current_avg_price]),  # 차원 조정: 1차원 배열로 변환
            "pnl": np.array([self.pnl]),  # 차원 조정: 1차원 배열로 변환
            "total_pnl": np.array([self.total_pnl]),  # 차원 조정: 1차원 배열로 변환
            "usdt_balance": np.array([round(self.usdt_balance, 2)]),  # 차원 조정: 1차원 배열로 변환
            "size": np.array([self.btc_size]),  # 차원 조정: 1차원 배열로 변환
            "margin": np.array([self.margin]),  # 차원 조정: 1차원 배열로 변환
            "total_balance": np.array([self.total_balance])  # 차원 조정: 1차원 배열로 변환
        }
        
        return obs
        
    def seed(self, seed=None):
        np.random.seed(seed)
        return [seed]
    
    # 나중에 수량지정을 위한 함수 (min_order부분만 바꾸면됌)
    def cac_order_size(self): 
        order_size = self.min_order
        return order_size
    
    # action을 수행할 수 있는 최소한의 조건 확인
    def act_check(self, action):
        required_margin = (self.cac_order_size() * self.current_price) / self.leverage
        
        if action == 0 or action == 1:
            if self.position == action or self.position == 2 or self.position is None:
                if self.usdt_balance > required_margin:
                    return action
                else:
                    return 3
            else:
                if self.usdt_balance + self.margin + self.pnl > required_margin:
                    return action
                else:
                    return 3
        
        elif action == 2:
            if self.position == 0 or self.position == 1:
                return 2
            else:
                return 3
        
        elif action == 3 or self.position is None:
            return 3

        

    # 포지션 진입
    def open_position(self, action):
        order_size = self.cac_order_size()
        required_margin = (order_size * self.current_price) / self.leverage
        open_fee = order_size * self.current_price * self.fee
        
        self.usdt_balance -= required_margin + open_fee
        self.btc_size += order_size
        self.margin += required_margin
        
        self.order_price = order_size * self.current_price
        self.current_avg_price = (self.order_price + self.last_size_value) / self.btc_size
        self.last_size_value = self.btc_size * self.current_price
                        
        self.pnl = (1 if action == 0 else -1) * (
            self.current_price - self.current_avg_price) * self.btc_size * self.leverage
            
        self.total_fee -= open_fee
        self.total_balance = self.usdt_balance + self.margin
        self.position = action
        
    def close_position(self):
        closing_fee = self.btc_size * self.current_price * self.fee
        closing_pnl = (1 if self.position == 0 else -1) * (
            self.current_price - self.current_avg_price) * self.btc_size * self.leverage
        
        self.usdt_balance += self.margin + closing_pnl - closing_fee
        self.total_fee -= closing_fee
        self.total_pnl += closing_pnl
        self.closing_pnl = closing_pnl
        
        self.btc_size = 0
        self.margin = 0
        self.pnl = 0
        self.last_size_value = 0
        
        self.total_balance = self.usdt_balance + self.margin
        self.position = 2

    def act(self, action):
        action = self.act_check(action)
        if action == 0 or action == 1:  # Long or Short
            if self.position == action or self.position == 2 or self.position is None:
                self.open_position(action)
            else:
                self.close_position()
                self.open_position(action)
        
        elif action == 2:  # Close
            if self.position == 0 or self.position == 1:
                self.close_position()
                
        
        elif action == 3:  # Hold
            if self.position == 0:  # Long
                self.pnl = (self.current_price - self.current_avg_price) * self.btc_size * self.leverage
            elif self.position == 1:  # Short
                self.pnl = (self.current_avg_price - self.current_price) * self.btc_size * self.leverage
            
            self.total_balance = self.usdt_balance + self.margin
        return action
    
    '''
    액션 :
        action : 0=Long, 1=Short, 2=Close, 3=Hold
        position : 0=Long, 1=Short, 2=None
        ex) (0.002*68000)/1=136, (0.002*68000)/2=68 필요 증거금 계산 예시 #
        
        return : self.position, self.acutal_action, self.pnl, self.closing_pnl, self.total_pnl, self.total_balance
        
        주문 수량은 일단 항상 최소 주문 금액으로 하겠습니다.
        최소 수량으로 해도 0.002개 이고 1배율일 경우 증거금 136usdt 정도 들어갑니다.
        
        추후 수량이 커질시 미결손실 또한 고려해야함
    ''' 
    
    # df 데이터를 받아 full_window_size만큼 랜덤 위치로 잘라서 Obs_df와 train_df로 나눈 df를 반환해주는 함수
    def generate_random_data_slice(df, full_window_size, test_window_size):
        strat_index = np.random.randint(0, len(df) - full_window_size)
        end_index = strat_index + full_window_size
        obs_end = end_index - test_window_size
        
        slice_df = df[strat_index:end_index]
        obs_df = df[strat_index:obs_end]
        train_df = df[obs_end:end_index]
        # slice_df : 전체 데이터에서 랜덤한 위치로 잘라낸 데이터
        # obs_df : 전체 데이터중 현재 스텝의 이전 데이터로써 이미 알고있는 차트 데이터
        # train_df : 전체 데이터중 현재 스텝의 이후 데이터로써 학습을 위한 차트 데이터
        
        return slice_df, obs_df, train_df
    
    # 다음 step과 가격을 가져옴
    def next_obs(self): 
        row_to_move = self.train_df.iloc[0:1] # train_df의 첫 행을 가져옴
        self.obs_df = pd.concat([self.obs_df, row_to_move]) # obs_df의 마지막 행에 train_df의 첫 행을 추가
        self.train_df = self.train_df.drop(self.train_df.index[0]) # train_df의 첫 행을 제거 (메모리 절약을 위해)
        self.current_step = self.obs_df.tail(1) # obs_df의 마지막 행을 현재 스텝으로 설정
        self.current_price = round(random.uniform(self.current_step['Open'].iloc[-1], self.current_step['Close'].iloc[-1]), 2) # 현재 가격을 시가, 종가 사이 랜덤 값으로 결정

    def calculate_reward(self):
        reward = 0
        if self.closing_pnl > 0:
            reward += 2
            
        elif self.closing_pnl < 0:
            reward -= 1
            
        self.closing_pnl = 0
        return reward
    
    def step(self, action):
        
        self.next_obs() # 다음 obs를 가져옴
        if (np.array(self.current_step) == None).all(): # 다음 훈련 데이터 없을 시 done = True로 변경 종료
            done = True
            return None, None, done, {}
        
        action = self.act(action) # action을 수행함.
        reward = self.calculate_reward() # reward 계산
        action_row = pd.DataFrame({'action': [action]}, index=[self.slice_df.index[self.current_index]]) 
        self.action_history = pd.concat([self.action_history, action_row]) # action_history에 action 추가
        if self.total_balance < self.total_balance * 0.3:
            done = True
        else:
            done = False

        # obs = {
        #     "chart_data": self.current_step.values.flatten(),
        #     "position": self.position,
        #     "action": action,
        #     "current_price": self.current_price,
        #     "avg_price": self.current_avg_price,
        #     "pnl": self.pnl,
        #     "total_pnl": self.total_pnl,
        #     "usdt_balance": round(self.usdt_balance, 2),
        #     "size": self.btc_size,
        #     "margin": self.margin,
        #     "total_balance": self.total_balance
        # }
        obs = {
            "chart_data": self.current_step.values,  # 차원 조정: 2차원 배열로 변환
            "position": np.array([self.position]),  # 차원 조정: 1차원 배열로 변환
            "action": np.array([action]),  # Hold 액션으로 초기화, 차원 조정: 1차원 배열로 변환
            "current_price": np.array([self.current_price]),  # 차원 조정: 1차원 배열로 변환
            "avg_price": np.array([self.current_avg_price]),  # 차원 조정: 1차원 배열로 변환
            "pnl": np.array([self.pnl]),  # 차원 조정: 1차원 배열로 변환
            "total_pnl": np.array([self.total_pnl]),  # 차원 조정: 1차원 배열로 변환
            "usdt_balance": np.array([round(self.usdt_balance, 2)]),  # 차원 조정: 1차원 배열로 변환
            "size": np.array([self.btc_size]),  # 차원 조정: 1차원 배열로 변환
            "margin": np.array([self.margin]),  # 차원 조정: 1차원 배열로 변환
            "total_balance": np.array([self.total_balance])  # 차원 조정: 1차원 배열로 변환
        }


        return obs, reward, done, {}

In [66]:
#self, df, full_window_size, test_window_size, usdt_balance=1000, btc_size=0, leverage=1
full_window_size = 100
test_window_size = 60
usdt_balance = 1000
btc_size = 0
leverage = 1
env = stablebaselineEnv(df, full_window_size, test_window_size, usdt_balance, btc_size, leverage)
# env.reset()

In [ ]:
env.reset()

In [68]:
env.current_step.values.shape

(1, 5)

In [60]:
action = 1
obs, reward, done, _ = env.step(action)
obs, reward, done

({'chart_data': array([4.709086e+04, 4.710040e+04, 4.703601e+04, 4.704219e+04,
         2.842520e+01]),
  'position': array([1]),
  'action': array([1]),
  'current_price': array([47061.4]),
  'avg_price': array([47061.4]),
  'pnl': array([-0.]),
  'total_pnl': array([0]),
  'usdt_balance': array([905.83]),
  'size': array([0.002]),
  'margin': array([94.1228]),
  'total_balance': array([999.9529386])},
 0,
 False)

In [70]:
# 환경 생성
env = stablebaselineEnv(df, full_window_size=200, test_window_size=120, usdt_balance=1000, btc_size=0, leverage=1)

# 벡터화된 환경 생성 (병렬 학습을 위해)
vec_env = make_vec_env(lambda: env, n_envs=4)

# 모델 생성
# model = PPO("MlpPolicy", vec_env, verbose=1)
model = PPO("MultiInputPolicy", vec_env, verbose=1)

# 체크포인트 콜백 설정
checkpoint_callback = CheckpointCallback(save_freq=10000, save_path='./checkpoints/', name_prefix='rl_model')

# 모델 학습
model.learn(total_timesteps=1000, callback=checkpoint_callback)

# 학습된 모델 저장
model.save("ppo_stablebaseline")

# 학습된 모델 로드
loaded_model = PPO.load("ppo_stablebaseline")

# 테스트
obs = env.reset()
while True:
    action, _states = loaded_model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        break

Using cpu device


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
pip install shimmy>=0.2.1